## 提供されているモデル

* https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b
* https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast
* https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-instruct
* https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct


## 提供されているデモ用 Spaces
* [ELYZA-japanese-Llama-2-7b-instruct](https://huggingface.co/spaces/elyza/ELYZA-japanese-Llama-2-7b-instruct-demo-1)
* [ELYZA-japanese-Llama-2-7b-fast-instruct](https://huggingface.co/spaces/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct-demo-1)

## ELYZA-japanese-Llama-2-7b
Baseのモデルを試してみる。

Base Model: https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b

In [ ]:
!pip install -q transformers accelerate sentencepiece bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00


In [ ]:
# テキストが見やすいようにwrapしておく
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, #adds speed with minimal loss of quality.
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained("elyza/ELYZA-japanese-Llama-2-7b")
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b",
    device_map="auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from pprint import pprint

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
text = "坂本龍馬は、"

prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)

pprint(prompt)

'<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。\n<</SYS>>\n\n坂本龍馬は、 [/INST] '


In [ ]:
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
print(output)

<span style="color: #0000FF;">あなたは誠実で優秀な日本人のアシスタントです。
</span>
<span style="color: #0000FF;">あなたは誠実で優秀な日本人のアシスタントです。
</span> nobody 2018/03/12 12:00 あなたは誠実で優秀な日本人のアシスタントです。
あなたは誠実で優秀な日本人のアシスタントです。
あなたは誠実で優秀な日本人のアシスタントです。
あなたは誠実で優秀な日本人のアシスタントです。
あなたは誠実で優秀な日本人のアシスタントです。
あなたは誠


In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
tokenizer.pad_token_id, tokenizer.eos_token_id

(2, 2)

In [ ]:
tokenizer.vocab_size

32000

In [ ]:
with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


<s> 坂本龍馬は、幕末の日本を代表する志士であり、「独立自由」という言葉を生み出しました。 Unterscheidung von Kreditkarten und Geldkarte. 1596年に完成されたルネサンス期の最大の作品だが、これ以前のバロック時代の音楽から見え


In [ ]:
# model.to(torch.device('cpu'))
# del model
# torch.cuda.empty_cache()

## ELYZA-japanese-Llama-2-7b-instruct
早速 Instruct 版を試してみたいと思います。

* https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-instruct

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

model_name = 'elyza/ELYZA-japanese-Llama-2-7b-instruct'

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer.vocab_size

32000

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

def build_prompt(text:str, system_prompt:str | None = None) -> str:
    system_prompt = system_prompt or DEFAULT_SYSTEM_PROMPT
    prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{system_prompt}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
    )
    return prompt

build_prompt("1 + 1 は何ですか？")

'<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。\n<</SYS>>\n\n1 + 1 は何ですか？ [/INST] '

In [ ]:
text = """
りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
""".strip()
inputs = tokenizer(build_prompt(text), add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？ [/INST]  りんごが5つあるところから、2つのりんごを取り除いた場合、残ったりんごの数は3つです。</s>


In [ ]:
text = """
バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
""".strip()
inputs = tokenizer(build_prompt(text), add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？ [/INST]  バットがボールよりも1000円高いところから考えます。

バットとボールの合計金額は、1100円 - 1000円 = 100円です。

よって、ボールは1100円-100=1000円です。</s>


In [ ]:
text = """
引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
""".strip()
inputs = tokenizer(build_prompt(text), add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.01,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], )
print(output)


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。 [/INST]  承知しました。フィボナッチ数列におけるk個目の値を求めるPython関数を以下に記述します: 

```python
def fib_k(k):
    if k == 0 or k == 1:
        return k
    else:
        return fib_k(k-1) + fib_k(k-2)
```</s>


In [ ]:
text = """
次の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
""".strip()
inputs = tokenizer(build_prompt(text), add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

次の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total? [/INST]  承知しました。翻訳は以下のようになります。

日本語: 3つのリンゴと2つのレモンがあった。そのものすべての果物の数は、どれくらいですか？</s>


## ELYZA-japanese-Llama-2-7b-fast-instruct

ELYZA様のブログ記事より：

> Llama 2に日本語の語彙を追加して事前学習を行ったモデルです。元のLlama 2は日本語の語彙が少ないため、日本語の文章を表すのに要するトークン数が英語と比べて多く、入出力できる文章量が短くなってしまうとともに、推論速度が遅いという問題を抱えていました。

> そこで今回新たに13,042個の日本語の語彙を追加し、結果として同じ日本語の文章を表すのに必要なトークン数を、約55%まで削減することができました。推論速度に換算すると約1.82倍となっており、大幅な効率化に成功しています。また、ELYZA-japanese-Llama-2-7b-fastに対して事後学習を行った「ELYZA-japanese-Llama-2-7b-fast-instruct」も同時に公開しています。

Model: https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = 'elyza/ELYZA-japanese-Llama-2-7b-fast-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer.vocab_size

45043

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

def build_prompt(text:str, system_prompt:str | None = None) -> str:
    system_prompt = system_prompt or DEFAULT_SYSTEM_PROMPT
    prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{system_prompt}{E_SYS}",
        prompt=text,
        e_inst=E_INST,
    )
    return prompt

build_prompt("1 + 1 は何ですか？")

'<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。\n<</SYS>>\n\n1 + 1 は何ですか？ [/INST] '

In [ ]:
text = """
りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
""".strip()
inputs = tokenizer(build_prompt(text), add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？ [/INST]  りんごが5つあるとします。その場合、取り除いたりんごの数を10より差し引くことで、答えを求めることができます。

10 - (5 + 2) = 10 - 7 = 3

よって、残りのりんごの数は3個です。</s>


In [ ]:
text = """
バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
""".strip()
inputs = tokenizer(build_prompt(text), add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？ [/INST]  バットがボールより1000円高いので、ボールの値段はバットの値段から1000円引いた900円です。</s>


In [ ]:
text = """
引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
""".strip()
inputs = tokenizer(build_prompt(text), add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], )
print(output)


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。 [/INST]  承知しました。以下がPythonで定義されるフィボナッチ数列のk個目に関する関数です: 

```python
def fib_k(k):
    if k == 0 or k == 1:
        return k
    elif k % 2 == 0:
        return fib_k(k-1) + fib_k(k)
    else:
        return fib_k(k-1) * 2 ** (k-1)
```</s>
